In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import lightgbm as lgb
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

# Step 1: Data Preprocessing
def load_and_preprocess_data(data_path, img_size):
    images = []
    labels = []
    classes = os.listdir(data_path)
    class_map = {cls: idx for idx, cls in enumerate(classes)}

    for cls in classes:
        cls_path = os.path.join(data_path, cls)
        for img_file in os.listdir(cls_path):
            img = cv2.imread(os.path.join(cls_path, img_file))
            img = cv2.resize(img, img_size)
            images.append(img)
            labels.append(class_map[cls])

    images = np.array(images, dtype='float32') / 255.0
    labels = np.array(labels)
    return images, labels, class_map

def augment_data(images):
    augmented_images = []
    for img in images:
        augmented_images.append(cv2.flip(img, 1))  # Horizontal flip
        augmented_images.append(cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE))  # Rotate 90
        augmented_images.append(cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE))  # Rotate -90
    return np.array(augmented_images, dtype='float32')

# Step 2: Define Swin Transformer Blocks
class SwinTransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, dynamic_factor=1.0, **kwargs):
        super(SwinTransformerBlock, self).__init__(**kwargs)
        self.mha = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ln1 = LayerNormalization()
        self.ln2 = LayerNormalization()
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation='gelu'),
            Dense(embed_dim)
        ])
        self.dynamic_factor = dynamic_factor

    def call(self, inputs):
        # Dynamic adjustment of input weights
        dynamic_weights = self.dynamic_factor * tf.reduce_mean(inputs, axis=1, keepdims=True)
        inputs = inputs + dynamic_weights

        attn_output = self.mha(inputs, inputs)
        out1 = self.ln1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        return self.ln2(out1 + ffn_output)

def build_swin_transformer(input_shape, embed_dim, num_heads, ff_dim, num_blocks, dynamic_factor=1.0):
    inputs = Input(shape=input_shape)
    x = inputs
    for _ in range(num_blocks):
        x = SwinTransformerBlock(embed_dim, num_heads, ff_dim, dynamic_factor=dynamic_factor)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    return Model(inputs, x)

# Step 3: Classification using LightGBM
def train_lightgbm(features, labels):
    params = {
        'objective': 'multiclass',
        'num_class': len(np.unique(labels)),
        'learning_rate': 0.1,
        'max_depth': 8,
        'num_leaves': 31,
        'metric': 'multi_logloss'
    }
    dataset = lgb.Dataset(features, label=labels)
    model = lgb.train(params, dataset)
    return model

# Step 4: Evaluation
def evaluate_model(model, test_features, test_labels):
    preds = model.predict(test_features)
    preds_labels = np.argmax(preds, axis=1)

    acc = accuracy_score(test_labels, preds_labels)
    precision = precision_score(test_labels, preds_labels, average='weighted')
    recall = recall_score(test_labels, preds_labels, average='weighted')
    f1 = f1_score(test_labels, preds_labels, average='weighted')

    return acc, precision, recall, f1

# Step 5: Grad-CAM for Interpretability
def grad_cam(model, img, layer_name):
    grad_model = Model([model.inputs], [model.get_layer(layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(np.expand_dims(img, axis=0))
        class_idx = np.argmax(predictions[0])
        loss = predictions[:, class_idx]

    grads = tape.gradient(loss, conv_outputs)[0]
    weights = tf.reduce_mean(grads, axis=(0, 1))
    cam = np.dot(conv_outputs[0], weights)
    cam = np.maximum(cam, 0)
    cam = cam / np.max(cam)
    cam = cv2.resize(cam, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * cam)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    return heatmap

# Main Execution
if __name__ == '__main__':
    data_path = 'content/drive/mydrive/optomdb/'
    img_size = (224, 224)
    embed_dim = 64
    num_heads = 4
    ff_dim = 128
    num_blocks = 4
    dynamic_factor = 1.2

    images, labels, class_map = load_and_preprocess_data(data_path, img_size)
    images = augment_data(images)
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

    swin_model = build_swin_transformer((224, 224, 3), embed_dim, num_heads, ff_dim, num_blocks, dynamic_factor=dynamic_factor)
    train_features = swin_model.predict(X_train)
    test_features = swin_model.predict(X_test)

    lgb_model = train_lightgbm(train_features, y_train)
    acc, precision, recall, f1 = evaluate_model(lgb_model, test_features, y_test)

    print(f'Accuracy: {acc}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}')

    sample_image = X_test[0]
    heatmap = grad_cam(swin_model, sample_image, layer_name='global_average_pooling2d')
    plt.imshow(cv2.addWeighted(sample_image, 0.6, heatmap, 0.4, 0))
    plt.show()
